In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from lightgbm import LGBMRegressor

In [ ]:
df=pd.read_csv('/content/cas train.csv')

In [ ]:
df.head()

,id,Date,Count,Hour,Hotness,Humidity,Wind Speed,Visibility,Solar Radiation,Rainfall,Snowfall,Seasons,Holiday,Working Day
0,1,05/08/2018,222,6:00,300.3,83,5.04,1080,0.00,0.0,0.0,Summer,NO,YES
1,2,21/01/2018,170,23:00,272.6,32,2.16,2000,0.00,0.0,0.0,Winter,NO,YES
2,3,11/05/2018,486,6:00,285.0,83,1.44,321,0.01,0.0,0.0,Spring,NO,YES
3,4,05/07/2018,929,13:00,301.1,67,3.96,1772,1.43,0.0,0.0,Summer,NO,YES
4,5,02/01/2018,58,5:00,269.9,67,2.16,1384,0.00,0.0,0.0,Winter,NO,YES


In [ ]:
df.describe()

,id,Count,Hotness,Humidity,Wind Speed,Visibility,Solar Radiation,Rainfall,Snowfall
count,4986.000000,4986.000000,4986.000000,4986.000000,4986.000000,4986.000000,4986.000000,4986.000000,4986.00000
mean,2493.500000,705.851986,285.977778,58.247493,6.159495,1432.546731,0.576897,0.154673,0.06775
std,1439.478551,646.444092,11.838222,20.373339,3.679312,605.976182,0.874589,1.121118,0.40789
min,1.000000,0.000000,255.200000,0.000000,0.000000,33.000000,0.000000,0.000000,0.00000
25%,1247.250000,191.000000,276.700000,43.000000,3.240000,942.250000,0.000000,0.000000,0.00000
50%,2493.500000,505.000000,286.700000,57.000000,5.400000,1678.000000,0.010000,0.000000,0.00000
75%,3739.750000,1066.750000,295.400000,74.000000,8.280000,1999.000000,0.950000,0.000000,0.00000
max,4986.000000,3556.000000,312.400000,98.000000,26.280000,2000.000000,3.520000,29.500000,8.80000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986 entries, 0 to 4985
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4986 non-null   int64  
 1   Date             4986 non-null   object 
 2   Count            4986 non-null   int64  
 3   Hour             4986 non-null   object 
 4   Hotness          4986 non-null   float64
 5   Humidity         4986 non-null   int64  
 6   Wind Speed       4986 non-null   float64
 7   Visibility       4986 non-null   int64  
 8   Solar Radiation  4986 non-null   float64
 9   Rainfall         4986 non-null   float64
 10  Snowfall         4986 non-null   float64
 11  Seasons          4986 non-null   object 
 12  Holiday          4986 non-null   object 
 13  Working Day      4986 non-null   object 
dtypes: float64(5), int64(4), object(5)
memory usage: 545.5+ KB


In [ ]:
df.isnull().sum()

id                 0
Date               0
Count              0
Hour               0
Hotness            0
Humidity           0
Wind Speed         0
Visibility         0
Solar Radiation    0
Rainfall           0
Snowfall           0
Seasons            0
Holiday            0
Working Day        0
dtype: int64

In [ ]:



def prep(df):
    df = df.drop(['id'], axis=1)
    df['Date'] = df['Date'].apply(lambda x:datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d"))
    df['Date'] = pd.to_datetime(df['Date'])
    df['Hour'] = df['Hour'].apply(lambda x: x.split(':', 1)[0])
    df['Hour']=df['Hour'].astype(int)
    df['year'] = pd.to_datetime(df['Date']).dt.year
    df['month'] = pd.to_datetime(df['Date']).dt.month
    df['day'] = df['Date'].apply(lambda x : x.dayofweek)
    df['Seasons']= df['Seasons'].map({"Winter":1,"Summer":2,"Spring":3,"Autumn":4})
    df['Holiday'] = df['Holiday'].apply(lambda x: 1 if x=="YES" else 0)
    df['Working Day'] = df['Working Day'].apply(lambda x: 1 if x=="YES" else 0)
    return df

In [ ]:
df['Count'] = df['Count'].apply(lambda x: 1 if x<1.000 else x)
y = np.log(df['Count'])
x = prep(df)
x = x.drop(['Count'] , axis = 1)

In [ ]:
x

,Date,Hour,Hotness,Humidity,Wind Speed,Visibility,Solar Radiation,Rainfall,Snowfall,Seasons,Holiday,Working Day,year,month,day
0,2018-08-05,6,300.3,83,5.04,1080,0.00,0.0,0.0,2,0,1,2018,8,6
1,2018-01-21,23,272.6,32,2.16,2000,0.00,0.0,0.0,1,0,1,2018,1,6
2,2018-05-11,6,285.0,83,1.44,321,0.01,0.0,0.0,3,0,1,2018,5,4
3,2018-07-05,13,301.1,67,3.96,1772,1.43,0.0,0.0,2,0,1,2018,7,3
4,2018-01-02,5,269.9,67,2.16,1384,0.00,0.0,0.0,1,0,1,2018,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,2018-07-21,19,306.4,51,9.36,2000,0.59,0.0,0.0,2,0,1,2018,7,5
4982,2018-03-19,19,281.4,51,11.88,1977,0.01,0.0,0.0,3,0,1,2018,3,0
4983,2018-10-03,13,295.5,40,9.36,1854,2.64,0.0,0.0,4,1,1,2018,10,2
4984,2018-01-09,21,268.2,59,3.24,2000,0.00,0.0,0.0,1,0,1,2018,1,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [ ]:
model = RandomForestRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

TypeError: ignored

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
extra_tree = ExtraTreeRegressor(random_state=0)
reg = BaggingRegressor(extra_tree, random_state=0).fit(x_train, y_train)
reg.score(x_test, y_test)
y_pred = reg.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
mdl = RandomForestRegressor()
mdl.fit(x_train , y_train)
mdl.score(x_test , y_test)

In [ ]:
y_pred = mdl.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

regr = RandomForestRegressor()

parameters = {'n_estimators':[200] , 'min_samples_split':[2]}
clf = GridSearchCV(regr , parameters)

In [ ]:
clf.fit(x_train , y_train)

In [ ]:
regr = RandomForestRegressor(n_estimators=200 , max_depth=50 , min_samples_split=2)
regr.fit(x_train , y_train)
regr.score(x_test , y_test)

In [ ]:
df_test = pd.read_csv('/content/cas test.csv')
df_test_prep = prep(df_test)
y_pred = regr.predict(df_test_prep)
y_min = np.exp(y_pred)

In [ ]:
result = pd.DataFrame()
result['id']=df_test['id']
result['Count'] = np.round_(np.array(y_min), decimals=0, out=None)
result['Count'] = result['Count'].astype(int)
result.to_csv( 'result.csv' , index=False)